In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, random
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split

In [15]:
# Define IAM role
role = get_execution_role()
prefix = 'xgboost'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'hcp-ml-project' # <--- change this variable to a unique name for your bucket
bucket = bucket_name
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)


S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
#split the downloaded data into train/test/validation files
FILE_TRAIN = 'train'
FILE_VALIDATION = 'validation'
FILE_TEST = 'test'


In [5]:
def assign_weight(row):
    attach_label = row['WILL_ATTACH_IN_60']
    cc_count = row['CC_COUNT']
    if attach_label == 1 and cc_count > 0:
        return 0.5
    elif attach_label == 0 and cc_count <= 0:
        return 0.5
    else:
        return 1

def get_dataset():
    try:
        df = pd.read_csv('../cc_attach_project.csv',index_col=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    
    to_drop = ['ENROLLMENT_DATE', 'DATE']
    df.drop(to_drop, inplace=True, axis=1)
    # df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
    # df['DATE'] = pd.to_datetime(df['DATE'])
    df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
    df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
    df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
    df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
    df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
    df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
    df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
    # df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
    cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
    df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
    df['INDUSTRY'] = df['INDUSTRY'].astype('category')
    df['ORGSIZE'] = df['ORGSIZE'].abs()
    # df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?

    #convert to all floats
    # df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

    #org_id same means same org!!!
    weight = df.apply(assign_weight, axis=1)

    df.insert(1, 'weight', weight, allow_duplicates=False) #insert weight into the df
    df = df.sample(frac=1, replace=False, random_state=1) #Shuffle the df

    df = pd.get_dummies(df)
    
    print(df.shape)
    train, test = train_test_split(df, test_size=0.15, random_state=123)
    train, val = train_test_split(train, test_size=0.2, random_state=123)
    
    print(train.shape, test.shape, val.shape)
    
    return (train, test, val)

In [12]:

def write_to_s3(fobj, bucket, key):
    return boto3.Session(region_name=my_region).resource('s3').Bucket(bucket).Object(key).upload_file(fobj)

def upload_to_s3(partition_name, partition):
    num_partition = 4                                 # partition file into 5 parts
    partitions = np.array_split(partition, num_partition)
    for i in range(num_partition):
        filename = "{}-{}.csv".format(partition_name, str(i))
        partition.to_csv(filename, header=False, index=False)
        key = "{}/{}/{}".format(prefix,partition_name,filename)
        url = 's3n://{}/{}'.format(bucket, key)
        print('Writing to {}'.format(url))
        write_to_s3(filename, bucket, key)
        print('Done writing to {}'.format(url))

def download_from_s3(partition_name, number, filename):
    key = "{}/{}/{}".format(prefix,partition_name, number)
    url = 's3n://{}/{}'.format(bucket, key)
    print('Reading from {}'.format(url))
    s3 = boto3.resource('s3', region_name = my_region)
    s3.Bucket(bucket).download_file(key, filename)
    try:
        s3.Bucket(bucket).download_file(key, 'mnist.local.test')
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print('The object does not exist at {}.'.format(url))
        else:
            raise        

def upload_data():
    train, test, val = get_dataset()
    partitions = [(FILE_TRAIN, train), (FILE_VALIDATION, val), (FILE_TEST, test)]
    for partition_name, partition in partitions:
        print('{}: {}'.format(partition_name, partition.shape))
        upload_to_s3(partition_name, partition)


In [13]:
upload_data()

Success: Data loaded into dataframe.
(1205612, 172)
(819816, 172) (180842, 172) (204954, 172)
train: (819816, 172)
Writing to s3n://hcp-ml-project/xgboost/train/train-0.csv
Done writing to s3n://hcp-ml-project/xgboost/train/train-0.csv
Writing to s3n://hcp-ml-project/xgboost/train/train-1.csv
Done writing to s3n://hcp-ml-project/xgboost/train/train-1.csv
Writing to s3n://hcp-ml-project/xgboost/train/train-2.csv
Done writing to s3n://hcp-ml-project/xgboost/train/train-2.csv
Writing to s3n://hcp-ml-project/xgboost/train/train-3.csv
Done writing to s3n://hcp-ml-project/xgboost/train/train-3.csv
validation: (204954, 172)
Writing to s3n://hcp-ml-project/xgboost/validation/validation-0.csv
Done writing to s3n://hcp-ml-project/xgboost/validation/validation-0.csv
Writing to s3n://hcp-ml-project/xgboost/validation/validation-1.csv
Done writing to s3n://hcp-ml-project/xgboost/validation/validation-1.csv
Writing to s3n://hcp-ml-project/xgboost/validation/validation-2.csv
Done writing to s3n://hcp